In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [2]:
df = pd.read_csv('고민분류용df2.csv', encoding= 'utf-8')

In [10]:
data = []

df.text.apply(lambda x : data.append(x))

0        None
1        None
2        None
3        None
4        None
         ... 
17315    None
17316    None
17317    None
17318    None
17319    None
Name: text, Length: 17312, dtype: object

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
tfidf = TfidfVectorizer(max_features= 1000)

In [13]:
tfidf.fit(data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [14]:
data2 = tfidf.transform(data).toarray()

In [15]:
data2.shape

(17312, 1000)

# Deeplearning으로 분류

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data2, df['label'], test_size = 0.1, random_state = 42)

In [44]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


In [45]:
x_train.shape

(15588, 1000)

In [46]:
y_train.shape

(15588, 10)

In [53]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(512, activation ='relu', input_shape=(1000,)))
model.add(layers.Dense(256, activation= 'relu'))
model.add(layers.Dense(256, activation= 'relu'))
model.add(layers.Dense(64, activation= 'relu'))
model.add(layers.Dense(10, activation='softmax'))

In [54]:
model.compile(optimizer='adam',\
              loss='categorical_crossentropy',\
              metrics=['accuracy'])

In [55]:
history = model.fit(x_train, y_train,\
                    epochs=10, batch_size = 128,validation_split=0.2)

Train on 12470 samples, validate on 3118 samples
Epoch 1/10
12470/12470 [==============================] - ETA: 32s - loss: 2.3103 - acc: 0.09 - ETA: 7s - loss: 2.2927 - acc: 0.2406 - ETA: 4s - loss: 2.2643 - acc: 0.280 - ETA: 3s - loss: 2.2191 - acc: 0.289 - ETA: 2s - loss: 2.1466 - acc: 0.337 - ETA: 2s - loss: 2.0455 - acc: 0.386 - ETA: 1s - loss: 1.9363 - acc: 0.432 - ETA: 1s - loss: 1.8123 - acc: 0.476 - ETA: 1s - loss: 1.6934 - acc: 0.509 - ETA: 1s - loss: 1.5793 - acc: 0.544 - ETA: 1s - loss: 1.4747 - acc: 0.574 - ETA: 1s - loss: 1.3735 - acc: 0.603 - ETA: 0s - loss: 1.2882 - acc: 0.627 - ETA: 0s - loss: 1.2098 - acc: 0.651 - ETA: 0s - loss: 1.1439 - acc: 0.671 - ETA: 0s - loss: 1.0860 - acc: 0.687 - ETA: 0s - loss: 1.0343 - acc: 0.702 - ETA: 0s - loss: 0.9943 - acc: 0.714 - ETA: 0s - loss: 0.9515 - acc: 0.726 - ETA: 0s - loss: 0.9117 - acc: 0.737 - ETA: 0s - loss: 0.8749 - acc: 0.748 - ETA: 0s - loss: 0.8455 - acc: 0.757 - ETA: 0s - loss: 0.8185 - acc: 0.765 - ETA: 0s - loss: 0.

In [56]:
model.evaluate(x_test,y_test)

1732/1732 [==============================] - ETA:  - ETA:  - ETA:  - 0s 68us/step


[0.24146388441444683, 0.9509237875288684]

In [59]:
# 운동몸매 :1 , 부부 :2 , 연애 :3 , 외모 : 4 , 진로 :5 ,직장 :6, 이별 7 , 친구 :8, 심리 : 9

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data2, df['label'], test_size = 0.2, random_state = 42)

y_train= y_train.values
y_test= y_test.values

# AdaBoost로 분류

In [76]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)

In [78]:
clf.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

# RandomForest로 분류

In [19]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=1000)


In [20]:
forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [21]:
from sklearn import metrics 

In [85]:
y_c_pred= clf.predict(x_test)

In [22]:
y_f_pred = forest.predict(x_test)

In [23]:
print('랜덤포레스트정확도 :', metrics.accuracy_score(y_test, y_f_pred))

랜덤포레스트정확도 : 0.9754548079699682


In [233]:
print('랜덤포레스트정확도 :', metrics.accuracy_score(y_test, y_f_pred))
print('아다 부스트 정확도 :', metrics.accuracy_score(y_test, y_c_pred))


랜덤포레스트정확도 : 0.9719976905311778
아다 부스트 정확도 : 0.8279445727482679


* 랜덤포레스트 정확도 : 97%
* Deeplearning 정확도 : 95%
* LSTM 정확도 : 87 %
* 아다부스트 정확도 : 82 %

## 따라서 랜덤포레스트 모델로 정함

* 사용했던 TfIDf 값과 랜덤 포레스트 모델 저장

In [ ]:
import dill
dill.dump(tfidf, open("gomin_classif_tfidf.obj","wb"))
dill.dump(forest, open("gomin_classif_RF.obj", "wb"))

## 실제 적용

In [234]:
def pos_tag2(sentences):
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    sentence = " ".join(tagger.morphs(sentences))
    sentences_pos.append(sentence)
    
  
    return sentences_pos

sentence ='와이프가 요새 저한테 소홀해요'

sentence = pos_tag2(sentence)


test_sen = tfidf.transform(sentence).toarray()

a=forest.predict(test_sen)

print(a[0])

In [502]:
sentence ='와이프가 요새 저한테 소홀해요'

sentence = pos_tag2(sentence)

In [503]:
sentence

['와이프 가 요새 저 한테 소홀해요']

In [504]:
test_sen = tfidf.transform(sentence).toarray()

In [486]:
# 운동몸매 :1 , 부부 :2 , 연애 :3 , 외모 : 4 , 진로 :5 ,직장 :6, 이별 7 , 친구 :8, 심리 : 9

In [487]:
a=forest.predict(test_sen)

print(a[0])

9


In [488]:
clf.predict(test_sen)

array([3], dtype=int64)

In [489]:
model.predict_classes(test_sen)

array([9], dtype=int64)

In [501]:
model_test = dill.load(open("RF_gomin_categori.obj","rb"))

model_test.predict(test_sen)

### 사용결과 랜덤포레스트가 가장 잘 분류함